# راهنمای کامل کتابخانه Multi-Calendar Dimension

این Notebook راهنمای جامعی برای همه قابلیت‌های کتابخانه `multi-calendar-dimension` است.

## فهرست مطالب:
1. نصب و راه‌اندازی
2. تبدیل تاریخ بین تقویم‌ها
3. دریافت نام ماه‌ها
4. بررسی تعطیلات رسمی
5. دریافت رویدادها
6. اطلاعات تاریخ امروز
7. تولید Date Dimension
8. تولید بازه ماهانه
9. مثال‌های پیشرفته


## 1. نصب و راه‌اندازی

برای نصب کتابخانه از PyPI:


In [1]:
# !pip install multi-calendar-dimension
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
# Import basic converters
from multi_calendar_dimension import (
    jalali_to_gregorian,
    gregorian_to_jalali,
    jalali_to_hijri,
    gregorian_to_hijri,
    hijri_to_gregorian,
    hijri_to_jalali
)

print("✅ کتابخانه با موفقیت import شد!")


✅ کتابخانه با موفقیت import شد!


## 2. تبدیل تاریخ بین تقویم‌ها

### 2.1 شمسی به میلادی (Jalali to Gregorian)


In [2]:
# تبدیل نوروز 1403 به میلادی
jalali_year, jalali_month, jalali_day = 1403, 1, 1
greg_year, greg_month, greg_day = jalali_to_gregorian(jalali_year, jalali_month, jalali_day)

print(f"شمسی: {jalali_year}/{jalali_month:02d}/{jalali_day:02d}")
print(f"میلادی: {greg_year}/{greg_month:02d}/{greg_day:02d}")
print(f"نوروز 1403 = {greg_day} March {greg_year}")


شمسی: 1403/01/01
میلادی: 2024/03/20
نوروز 1403 = 20 March 2024


### 2.2 میلادی به شمسی (Gregorian to Jalali)


In [3]:
# تبدیل میلادی به شمسی
jy, jm, jd = gregorian_to_jalali(2024, 3, 20)
print(f"میلادی: 2024/03/20")
print(f"شمسی: {jy}/{jm:02d}/{jd:02d}")
print(f"20 March 2024 = {jd} فروردین {jy}")


میلادی: 2024/03/20
شمسی: 1403/01/01
20 March 2024 = 1 فروردین 1403


### 2.3 میلادی به هجری (Gregorian to Hijri)


In [4]:
# تبدیل میلادی به هجری (با الگوریتم Umm al-Qura)
from multi_calendar_dimension import get_hijri_month_name

hy, hm, hd = gregorian_to_hijri(2023, 7, 19)
month_name_en = get_hijri_month_name(hm, english=True)
month_name_fa = get_hijri_month_name(hm, english=False)

print(f"میلادی: 2023/07/19")
print(f"هجری (عددی): {hd}/{hm}/{hy}")
print(f"هجری (انگلیسی): {hd} {month_name_en} {hy}")
print(f"هجری (فارسی): {hd} {month_name_fa} {hy}")


میلادی: 2023/07/19
هجری (عددی): 1/1/1445
هجری (انگلیسی): 1 Muharram 1445
هجری (فارسی): 1 محرم 1445


### 2.4 شمسی به هجری و برعکس


In [5]:
# شمسی به هجری
hy, hm, hd = jalali_to_hijri(1400, 2, 1)
print(f"1403/01/01 (نوروز) = {hd}/{hm}/{hy} هجری")
print(f"ماه هجری: {get_hijri_month_name(hm, True)}")

print("\n" + "-"*50 + "\n")

# هجری به شمسی
jy, jm, jd = hijri_to_jalali(1445, 1, 1)
print(f"1445/01/01 (محرم) = {jd}/{jm}/{jy} شمسی")


1403/01/01 (نوروز) = 9/9/1442 هجری
ماه هجری: Ramadan

--------------------------------------------------

1445/01/01 (محرم) = 28/4/1402 شمسی


## 3. نام ماه‌های هجری

دریافت نام ماه‌های قمری به فارسی و انگلیسی:


In [6]:
from multi_calendar_dimension import HIJRI_MONTH_NAMES, HIJRI_MONTH_NAMES_EN

print("لیست کامل ماه‌های هجری:\n")
print(f"{'شماره':<8} {'نام فارسی':<20} {'نام انگلیسی':<20}")
print("="*50)

for month_num in range(1, 13):
    fa_name = HIJRI_MONTH_NAMES[month_num]
    en_name = HIJRI_MONTH_NAMES_EN[month_num]
    print(f"{month_num:<8} {fa_name:<20} {en_name:<20}")


لیست کامل ماه‌های هجری:

شماره    نام فارسی            نام انگلیسی         
1        محرم                 Muharram            
2        صفر                  Safar               
3        ربیع‌الاول           Rabi' al-Awwal      
4        ربیع‌الثانی          Rabi' al-Thani      
5        جمادی‌الاول          Jumada al-Ula       
6        جمادی‌الثانی         Jumada al-Thani     
7        رجب                  Rajab               
8        شعبان                Sha'ban             
9        رمضان                Ramadan             
10       شوال                 Shawwal             
11       ذی‌قعده              Dhu al-Qa'dah       
12       ذی‌حجه               Dhu al-Hijjah       


## 4. بررسی تعطیلات رسمی ایران

شامل تعطیلات شمسی (ثابت) و هجری (متغیر):


In [7]:
from multi_calendar_dimension import is_iranian_holiday

# بررسی چند تاریخ
test_dates = [
    (1403, 1, 1, "نوروز"),
    (1403, 4, 26, "عاشورا 1446"),
    (1403, 11, 22, "22 بهمن"),
    (1403, 5, 15, "روز عادی"),
]

print("بررسی تعطیلات:\n")
for jy, jm, jd, desc in test_dates:
    result = is_iranian_holiday(jy, jm, jd)
    status = "✅ تعطیل" if result['is_holiday'] else "❌ کاری"
    holiday_type = f"({result['holiday_type']})" if result['is_holiday'] else ""
    
    print(f"{jy}/{jm:02d}/{jd:02d} - {desc:<20} {status} {holiday_type}")


بررسی تعطیلات:

1403/01/01 - نوروز                ✅ تعطیل (persian)
1403/04/26 - عاشورا 1446          ✅ تعطیل (hijri)
1403/11/22 - 22 بهمن              ✅ تعطیل (persian)
1403/05/15 - روز عادی             ❌ کاری 


## 5. دریافت رویدادها

### 5.1 رویدادهای فارسی


In [8]:
from multi_calendar_dimension import persian_events, persian_events_en

# رویداد نوروز
event_fa = persian_events.get((1, 1))
event_en = persian_events_en.get((1, 1))

print("رویداد 1 فروردین:")
print(f"  فارسی: {event_fa}")
print(f"  English: {event_en}")

print("\nچند رویداد دیگر:")
sample_dates = [(1, 12), (3, 14), (11, 22)]
for m, d in sample_dates:
    if (m, d) in persian_events:
        print(f"  {m}/{d}: {persian_events_en[(m, d)]}")


رویداد 1 فروردین:
  فارسی: عید نوروز
  English: Nowruz (Persian New Year)

چند رویداد دیگر:
  1/12: Islamic Republic Day of Iran
  3/14: Demise of Imam Khomeini (R.A.), Great Leader of the Revolution and Founder of Islamic Republic of Iran (1989) – Election of Ayatollah Khamenei as Leader (1989)
  11/22: Victory of Islamic Revolution of Iran and Fall of Monarchy (1979)


### 5.2 رویدادهای هجری


In [9]:
from multi_calendar_dimension import hijri_events, hijri_events_en

# رویدادهای مهم هجری
important_hijri = [(1, 10), (2, 20), (9, 21), (10, 1)]

print("رویدادهای مهم هجری:\n")
for m, d in important_hijri:
    if (m, d) in hijri_events_en:
        month_name = get_hijri_month_name(m, True)
        event = hijri_events_en[(m, d)]
        print(f"{d} {month_name}: {event}")


رویدادهای مهم هجری:

10 Muharram: Ashura of Imam Hussein (A.S.)
20 Safar: Arbaeen of Imam Hussein (A.S.)
21 Ramadan: Martyrdom of Imam Ali (A.S.) (40 A.H.)
1 Shawwal: Eid al-Fitr (Festival of Breaking the Fast)


## 6. اطلاعات جامع یک تاریخ شمسی

دریافت همه اطلاعات یک تاریخ (شمسی، میلادی، هجری، رویدادها، تعطیلات):


In [10]:
from multi_calendar_dimension import get_all_holidays_for_jalali_date

# دریافت اطلاعات کامل برای عاشورا
info = get_all_holidays_for_jalali_date(1403, 4, 26)

print("اطلاعات کامل تاریخ 1403/04/26:\n")
print(f"📅 تاریخ شمسی: {info['jalali_date']}")
print(f"📅 تاریخ میلادی: {info['gregorian_date']}")
print(f"📅 تاریخ هجری: {info['hijri_date']}")
print(f"\n🎉 تعطیل رسمی: {'بله' if info['is_official_holiday'] else 'خیر'}")
print(f"   - تعطیل شمسی: {'بله' if info['is_persian_holiday'] else 'خیر'}")
print(f"   - تعطیل هجری: {'بله' if info['is_hijri_holiday'] else 'خیر'}")

if info['persian_event']:
    print(f"\n📌 رویداد شمسی: {info['persian_event']}")
if info['hijri_event']:
    print(f"📌 رویداد هجری: {info['hijri_event']}")


اطلاعات کامل تاریخ 1403/04/26:

📅 تاریخ شمسی: 1403/04/26
📅 تاریخ میلادی: 2024/07/16
📅 تاریخ هجری: 1446/01/10

🎉 تعطیل رسمی: بله
   - تعطیل شمسی: خیر
   - تعطیل هجری: بله

📌 رویداد شمسی: تاسیس شورای نگهبان
📌 رویداد هجری: عاشورای حسینی (ع)


## 7. تولید جدول Date Dimension

برای استفاده در Data Warehouse و BI:


In [11]:
from multi_calendar_dimension import DateDimensionGenerator, DateRangeConfig

print("✅ Import successful!")
print("   DateDimensionGenerator: برای تولید جدول Date Dimension کامل")
print("   DateDimensionConfig: برای تنظیمات generator")

✅ Import successful!
   DateDimensionGenerator: برای تولید جدول Date Dimension کامل
   DateDimensionConfig: برای تنظیمات generator


In [12]:
# نیاز به pandas دارد
import pandas as pd

# تنظیمات نمایش بهتر
pd.set_option('display.max_columns', None)

try:
    from multi_calendar_dimension import DateDimensionGenerator, DateDimensionConfig  # ✅ اصلاح شد
    
    # تولید جدول Date Dimension برای یک سال
    config = DateDimensionConfig(  # ✅ اصلاح شد
        start_year=1403,
        end_year=1403,
        include_events=True,
        include_holidays=True,
        include_week_calculations=True
    )
    
    generator = DateDimensionGenerator(config)
    df = generator.generate()
    
    print(f"✅ جدول Date Dimension تولید شد!")
    print(f"📊 تعداد سطرها: {len(df)}")
    print(f"📊 تعداد ستون‌ها: {len(df.columns)}")
    
    # نمایش نمونه
    display_cols = ['shamsi_date_title', 'miladi_date', 'shamsi_month_title', 'shamsi_is_holiday']
    print(df[display_cols].head())
    
except ImportError as e:
    print("⚠️ برای استفاده از DateDimensionGenerator باید pandas نصب باشد")
    print(f"Error: {e}")

✅ جدول Date Dimension تولید شد!
📊 تعداد سطرها: 366
📊 تعداد ستون‌ها: 66
  shamsi_date_title miladi_date shamsi_month_title  shamsi_is_holiday
0        1403/01/01  2024-03-20       فروردین 1403                  1
1        1403/01/02  2024-03-21       فروردین 1403                  1
2        1403/01/03  2024-03-22       فروردین 1403                  1
3        1403/01/04  2024-03-23       فروردین 1403                  1
4        1403/01/05  2024-03-24       فروردین 1403                  0


c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:337: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.sort_values(['shamsi_month_id', 'shamsi_date']).groupby('shamsi_month_id', group_keys=False).apply(self._assign_monthly_weeks)
c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:338: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby 

### 7.1 مثال پیشرفته: تولید Date Dimension برای چند سال

می‌توانید Date Dimension را برای محدوده سال‌های مختلف تولید کنید:


In [13]:
import pandas as pd
from multi_calendar_dimension import DateDimensionGenerator, DateDimensionConfig

config = DateDimensionConfig(
    start_year=1403,
    end_year=1403,
    include_events=True,
    include_holidays=True,
    include_week_calculations=True
)

generator = DateDimensionGenerator(config)
df = generator.generate()

print(f"✅ تولید شد! {len(df)} روز، {len(df.columns)} ستون")
print(df.head())

✅ تولید شد! 366 روز، 66 ستون
   shamsi_date miladi_date miladi_day_of_week_title shamsi_date_title  \
0     14030101  2024-03-20                Wednesday        1403/01/01   
1     14030102  2024-03-21                 Thursday        1403/01/02   
2     14030103  2024-03-22                   Friday        1403/01/03   
3     14030104  2024-03-23                 Saturday        1403/01/04   
4     14030105  2024-03-24                   Sunday        1403/01/05   

   shamsi_next_date  shamsi_month_id shamsi_month_title  \
0          14030102           140301       فروردین 1403   
1          14030103           140301       فروردین 1403   
2          14030104           140301       فروردین 1403   
3          14030105           140301       فروردین 1403   
4          14030106           140301       فروردین 1403   

  shamsi_month_of_year_title  shamsi_season_id shamsi_season_title  \
0                    فروردین            140301           بهار 1403   
1                    فروردین         

c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:337: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.sort_values(['shamsi_month_id', 'shamsi_date']).groupby('shamsi_month_id', group_keys=False).apply(self._assign_monthly_weeks)
c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:338: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby 

In [14]:
try:
    # مثال: تولید Date Dimension برای سال‌های 1400 تا 1403
    config = DateDimensionConfig(
        start_year=1400,
        end_year=1403,
        include_events=True,
        include_holidays=True,
        include_week_calculations=True
    )
    
    generator = DateDimensionGenerator(config)
    df = generator.generate()
    
    print(f"✅ Date Dimension برای {1403-1400+1} سال تولید شد!")
    print(f"📊 کل روزها: {len(df):,}")
    print(f"📊 ستون‌ها: {len(df.columns)}")
    
    # آمار سریع
    print(f"\n📈 آمار:")
    print(f"   تعطیلات: {df['shamsi_is_holiday'].sum():,} روز")
    print(f"   آخر هفته‌ها: {df['shamsi_is_weekend'].sum():,} روز")
    print(f"   روزهای کاری: {(~df['shamsi_is_holiday'] & ~df['shamsi_is_weekend']).sum():,} روز")
    
except Exception as e:
    print(f"خطا: {e}")


✅ Date Dimension برای 4 سال تولید شد!
📊 کل روزها: 1,461
📊 ستون‌ها: 66

📈 آمار:
   تعطیلات: 242 روز
   آخر هفته‌ها: 417 روز
   روزهای کاری: -1,908 روز


c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:337: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.sort_values(['shamsi_month_id', 'shamsi_date']).groupby('shamsi_month_id', group_keys=False).apply(self._assign_monthly_weeks)
c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:338: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby 

## 8. تولید بازه ماهانه

تولید لیست تمام روزهای یک ماه:


In [15]:
from multi_calendar_dimension import DateRangeGenerator, DateRangeConfig, CalendarType

print("✅ Import successful!")
print("   DateRangeGenerator: برای تولید بازه ماهانه")
print("   DateRangeConfig: برای تنظیمات generator")
print("   CalendarType: نوع تقویم (JALALI, GREGORIAN, HIJRI)")

✅ Import successful!
   DateRangeGenerator: برای تولید بازه ماهانه
   DateRangeConfig: برای تنظیمات generator
   CalendarType: نوع تقویم (JALALI, GREGORIAN, HIJRI)


In [16]:
try:
    from multi_calendar_dimension import DateRangeGenerator, DateRangeConfig, CalendarType
    
    # تولید بازه فروردین 1403
    # ابتدا config را می‌سازیم
    config = DateRangeConfig(
        calendar_type=CalendarType.JALALI,
        start_year=1400,
        start_month=2,  # فروردین
        end_year=1400,
        end_month=2,    # فقط فروردین
        include_events=True,
        include_holidays=True
    )
    
    # حالا generator را با config می‌سازیم
    generator = DateRangeGenerator(config)
    df = generator.generate()
    
    print(f"✅ بازه فروردین 1403 تولید شد!")
    print(f"📊 تعداد روزها: {len(df)}")
    print(f"\n5 روز اول فروردین:")
    display_cols = ['shamsi_date_title', 'miladi_date', 'shamsi_is_holiday']
    print(df[display_cols].head())
    
    # خلاصه اطلاعات
    summary = generator.get_summary()
    print(f"\n📊 خلاصه:")
    print(f"   تعداد کل روزها: {summary['total_days']}")
    print(f"   تعداد تعطیلات: {summary['holidays_count']}")
    print(f"   تعداد آخر هفته‌ها: {summary['weekends_count']}")
    
except ImportError as e:
    print("⚠️ برای استفاده از DateRangeGenerator باید pandas نصب باشد")
    print(f"Error: {e}")


✅ بازه فروردین 1403 تولید شد!
📊 تعداد روزها: 31

5 روز اول فروردین:
   shamsi_date_title miladi_date  shamsi_is_holiday
31        1400/02/01  2021-04-21                  0
32        1400/02/02  2021-04-22                  0
33        1400/02/03  2021-04-23                  1
34        1400/02/04  2021-04-24                  0
35        1400/02/05  2021-04-25                  0

📊 خلاصه:
   تعداد کل روزها: 31
   تعداد تعطیلات: 5
   تعداد آخر هفته‌ها: 10


c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:337: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.sort_values(['shamsi_month_id', 'shamsi_date']).groupby('shamsi_month_id', group_keys=False).apply(self._assign_monthly_weeks)
c:\Users\pc\Desktop\pouya\ad_hoc\.venv\Lib\site-packages\multi_calendar_dimension\generator\dimension.py:338: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby 

In [17]:
df[df['shamsi_date']==14000223]

,shamsi_date,miladi_date,miladi_day_of_week_title,shamsi_date_title,shamsi_next_date,shamsi_month_id,shamsi_month_title,shamsi_month_of_year_title,shamsi_season_id,shamsi_season_title,shamsi_half_year_id,shamsi_half_year_title,shamsi_year_id,shamsi_month_of_year_id,shamsi_day_of_month_id,shamsi_day_of_year_id,shamsi_day_of_week_title,shamsi_day_of_week_id,shamsi_is_holiday,shamsi_is_happy_holiday,shamsi_is_sad_holiday,shamsi_is_weekend,gregorian_next_date,gregorian_month_id,gregorian_month_title,gregorian_month_of_year_title,gregorian_season_id,gregorian_season_title,gregorian_half_year_id,gregorian_half_year_title,gregorian_year_id,gregorian_month_of_year_id,gregorian_day_of_month_id,gregorian_day_of_year_id,gregorian_day_of_week_title,gregorian_day_of_week_id,gregorian_is_holiday,gregorian_is_happy_holiday,gregorian_is_sad_holiday,gregorian_is_weekend,hijri_date,hijri_date_title,hijri_month_id,hijri_month_title,hijri_month_of_year_title,hijri_year_id,hijri_month_of_year_id,hijri_day_of_month_id,hijri_is_holiday,hijri_is_happy_holiday,hijri_is_sad_holiday,shamsi_event_name,gregorian_event_name_en,gregorian_event_name_fa,hijri_event_name,hijri_event_name_en,week_num_in_month,shamsi_week_id,week_num_in_year,shamsi_week_title,shamsi_week_of_year_id,gregorian_week_id,gregorian_week_title,gregorian_week_of_year_id,miladi_week_id,miladi_week_num_in_month
53,14000223,2021-05-13,Thursday,1400/02/23,14000224,140002,اردیبهشت 1400,اردیبهشت,140002,بهار 1400,140002,نیمسال اول 1400,1400,2,23,54,پنج‌شنبه,6,0,0,0,1,2021-05-14,202105,May 2021,May,202105,Spring 2021,202105,H1 2021,2021,5,13,133,Thursday,6,0,0,0,0,14421001,1442/10/01,144210,شوال 1442,شوال,1442,10,1,1,1,0,None,None,None,عید فطر,Eid al-Fitr (Festival of Breaking the Fast),4,140002_4,8,هفته 8 سال 1400,1400_08,19,Week 19 2021,2021_19,19,3


## 9. مثال‌های کاربردی

### مثال 1: پیدا کردن تمام تعطیلات یک ماه


In [18]:
# پیدا کردن تمام تعطیلات فروردین 1403
year, month = 1403, 1

print(f"تعطیلات رسمی {month} / {year}:\n")

for day in range(1, 32):  # فروردین 31 روزه
    result = is_iranian_holiday(year, month, day)
    if result['is_holiday']:
        print(f"  {year}/{month:02d}/{day:02d} - تعطیل {result['holiday_type']}")


تعطیلات رسمی 1 / 1403:

  1403/01/01 - تعطیل persian
  1403/01/02 - تعطیل persian
  1403/01/03 - تعطیل persian
  1403/01/04 - تعطیل persian
  1403/01/12 - تعطیل persian
  1403/01/13 - تعطیل persian
  1403/01/22 - تعطیل hijri
  1403/01/23 - تعطیل hijri


### مثال 2: تبدیل چند تاریخ به صورت دسته‌ای


In [19]:
# تبدیل چند تاریخ میلادی به هجری با نام ماه
gregorian_dates = [
    (2023, 7, 19, "Islamic New Year"),
    (2024, 3, 11, "Ramadan Start"),
    (2024, 4, 10, "Eid al-Fitr"),
    (2024, 6, 17, "Eid al-Adha"),
]

print("تبدیل دسته‌ای میلادی به هجری:\n")
for gy, gm, gd, description in gregorian_dates:
    hy, hm, hd = gregorian_to_hijri(gy, gm, gd)
    month_name = get_hijri_month_name(hm, True)
    
    print(f"{description}:")
    print(f"  {gy}-{gm:02d}-{gd:02d} → {hd} {month_name} {hy}")
    print()


تبدیل دسته‌ای میلادی به هجری:

Islamic New Year:
  2023-07-19 → 1 Muharram 1445

Ramadan Start:
  2024-03-11 → 1 Ramadan 1445

Eid al-Fitr:
  2024-04-10 → 1 Shawwal 1445

Eid al-Adha:
  2024-06-17 → 11 Dhu al-Hijjah 1445



### مثال 3: بررسی سال کبیسه


In [20]:
from multi_calendar_dimension import is_leap_year_persian, is_hijri_leap

# بررسی سال‌های کبیسه شمسی
print("سال‌های کبیسه شمسی:")
for year in range(1403, 1408):
    leap = "کبیسه ✅" if is_leap_year_persian(year) else "عادی"
    print(f"  {year}: {leap}")

print("\nسال‌های کبیسه هجری:")
for year in range(1445, 1450):
    leap = "کبیسه ✅" if is_hijri_leap(year) else "عادی"
    print(f"  {year}: {leap}")


سال‌های کبیسه شمسی:
  1403: کبیسه ✅
  1404: عادی
  1405: عادی
  1406: عادی
  1407: کبیسه ✅

سال‌های کبیسه هجری:
  1445: کبیسه ✅
  1446: عادی
  1447: کبیسه ✅
  1448: عادی
  1449: عادی


### مثال 4: تقویم یک ماه با تعطیلات


In [21]:
# نمایش تقویم فروردین 1403 با تعطیلات
print("تقویم فروردین 1403 (با علامت‌گذاری تعطیلات):\n")

for day in range(1, 32):
    result = is_iranian_holiday(1403, 1, day)
    marker = "🔴" if result['is_holiday'] else "⚪"
    holiday_info = f" ({result['holiday_type']})" if result['is_holiday'] else ""
    
    # دریافت رویداد اگر وجود دارد
    event = persian_events.get((1, day), "")
    event_short = event[:30] + "..." if len(event) > 30 else event
    
    print(f"{marker} {1403}/{1:02d}/{day:02d}  {event_short}{holiday_info}")


تقویم فروردین 1403 (با علامت‌گذاری تعطیلات):

🔴 1403/01/01  عید نوروز (persian)
🔴 1403/01/02  عید نوروز - آغاز عملیات فتح ال... (persian)
🔴 1403/01/03  عید نوروز - روز جهانی هواشناسی (persian)
🔴 1403/01/04  عید نوروز (persian)
⚪ 1403/01/05  
⚪ 1403/01/06  روز امید، روز شادباش نویسی
⚪ 1403/01/07  
⚪ 1403/01/08  
⚪ 1403/01/09  
⚪ 1403/01/10  
⚪ 1403/01/11  
🔴 1403/01/12  روز جمهوری اسلامی ایران (persian)
🔴 1403/01/13  روز طبیعت (persian)
⚪ 1403/01/14  
⚪ 1403/01/15  
⚪ 1403/01/16  
⚪ 1403/01/17  
⚪ 1403/01/18  روز سلامتی (روز جهانی بهداشت)
⚪ 1403/01/19  شهادت آیت الله سید محمد باقر ص...
⚪ 1403/01/20  روز ملی فناوری هسته‌ای
⚪ 1403/01/21  شهادت امیر سپهبد علی صیاد شیرا...
🔴 1403/01/22   (hijri)
🔴 1403/01/23   (hijri)
⚪ 1403/01/24  
⚪ 1403/01/25  روز بزرگداشت عطار نیشابوری
⚪ 1403/01/26  
⚪ 1403/01/27  
⚪ 1403/01/28  
⚪ 1403/01/29  روز ارتش جمهوری اسلامی ایران
⚪ 1403/01/30  روز وقف
⚪ 1403/01/31  


### مثال 5: کار با DataFrame (pandas)


In [22]:
try:
    import pandas as pd
    
    # تبدیل لیست تاریخ‌های میلادی به DataFrame
    gregorian_dates = [
        "2024-03-20",
        "2024-07-19",
        "2024-12-25",
    ]
    
    # ایجاد DataFrame
    data = []
    for date_str in gregorian_dates:
        gy, gm, gd = map(int, date_str.split('-'))
        jy, jm, jd = gregorian_to_jalali(gy, gm, gd)
        hy, hm, hdd = gregorian_to_hijri(gy, gm, gd)
        
        data.append({
            'Gregorian': date_str,
            'Jalali': f'{jy}/{jm:02d}/{jd:02d}',
            'Hijri': f'{hy}/{hm:02d}/{hdd:02d}',
            'Hijri_Month': get_hijri_month_name(hm, True)
        })
    
    df = pd.DataFrame(data)
    print("📊 DataFrame با تاریخ‌های تبدیل شده:\n")
    print(df.to_string(index=False))
    
except ImportError:
    print("⚠️ pandas نصب نیست")


📊 DataFrame با تاریخ‌های تبدیل شده:

 Gregorian     Jalali      Hijri     Hijri_Month
2024-03-20 1403/01/01 1445/09/10         Ramadan
2024-07-19 1403/04/29 1446/01/13        Muharram
2024-12-25 1403/10/05 1446/06/23 Jumada al-Thani


### مثال 6: محاسبه فاصله بین دو تاریخ


In [23]:
from datetime import datetime

# تبدیل تاریخ شمسی به datetime
date1_jalali = (1400, 2, 1)
date2_jalali = (1400, 2, 29)

# تبدیل به میلادی
date1_greg = jalali_to_gregorian(*date1_jalali)
date2_greg = jalali_to_gregorian(*date2_jalali)

# محاسبه فاصله
dt1 = datetime(*date1_greg)
dt2 = datetime(*date2_greg)
diff = (dt2 - dt1).days

print(f"فاصله بین {date1_jalali} و {date2_jalali}:")
print(f"  {diff} روز")
print(f"  {diff / 30:.1f} ماه تقریبی")
print(f"  {diff / 365:.2f} سال تقریبی")


فاصله بین (1400, 2, 1) و (1400, 2, 29):
  28 روز
  0.9 ماه تقریبی
  0.08 سال تقریبی


### مثال 7: تبدیل تاریخ امروز به همه تقویم‌ها


In [24]:
# تاریخ امروز
today = datetime.now()
gy, gm, gd = today.year, today.month, today.day

# تبدیل به شمسی
jy, jm, jd = gregorian_to_jalali(gy, gm, gd)

# تبدیل به هجری
hy, hm, hdd = gregorian_to_hijri(gy, gm, gd)
hijri_month_name = get_hijri_month_name(hm, True)

# بررسی تعطیل
holiday_info = is_iranian_holiday(jy, jm, jd)

print("📅 امروز:\n")
print(f"میلادی: {gy}/{gm:02d}/{gd:02d}")
print(f"شمسی: {jy}/{jm:02d}/{jd:02d}")
print(f"هجری: {hdd} {hijri_month_name} {hy}")
print(f"\nتعطیل: {'بله ✅' if holiday_info['is_holiday'] else 'خیر ❌'}")


📅 امروز:

میلادی: 2025/10/28
شمسی: 1404/08/06
هجری: 6 Jumada al-Ula 1447

تعطیل: خیر ❌


### مثال 8: یافتن رویدادهای مهم یک سال


In [25]:
# یافتن همه تعطیلات رسمی سال 1403
from multi_calendar_dimension import persian_holidays, hijri_official_holidays

print("تعطیلات رسمی سال 1403:\n")

# تعطیلات شمسی (ثابت)
print("1️⃣ تعطیلات شمسی (ثابت):")
for (m, d), value in sorted(persian_holidays.items()):
    event = persian_events.get((m, d), "")
    print(f"  {m:02d}/{d:02d} - {event[:40]}")

# تعطیلات هجری (متغیر - باید برای هر سال محاسبه شوند)
print("\n2️⃣ تعطیلات هجری (برای سال 1403 شمسی):")
print("  نیاز به محاسبه تبدیل برای هر روز...")
print("  (در Date Dimension کامل محاسبه می‌شود)")


تعطیلات رسمی سال 1403:

1️⃣ تعطیلات شمسی (ثابت):
  01/01 - عید نوروز
  01/02 - عید نوروز - آغاز عملیات فتح المبین (1361
  01/03 - عید نوروز - روز جهانی هواشناسی
  01/04 - عید نوروز
  01/12 - روز جمهوری اسلامی ایران
  01/13 - روز طبیعت
  03/14 - رحلت حضرت امام خمینی (ره) رهبر کبیر انقل
  03/15 - قیام خونین 15 خرداد (1342) - روز جهانی م
  11/22 - پیروزی انقلاب اسلامی ایران و سقوط نظام ش
  12/29 - روز ملی شدن صنعت نفت ایران (1329)

2️⃣ تعطیلات هجری (برای سال 1403 شمسی):
  نیاز به محاسبه تبدیل برای هر روز...
  (در Date Dimension کامل محاسبه می‌شود)


## 10. خلاصه و نکات مهم

### تمام قابلیت‌های کتابخانه:


In [26]:
print("📚 کتابخانه Multi-Calendar Dimension\n")
print("="*60)

print("\n✅ تبدیل تاریخ:")
print("  • jalali_to_gregorian()")
print("  • gregorian_to_jalali()")
print("  • jalali_to_hijri()")
print("  • gregorian_to_hijri()")
print("  • hijri_to_gregorian()")
print("  • hijri_to_jalali()")

print("\n✅ نام ماه‌ها:")
print("  • get_hijri_month_name()")
print("  • HIJRI_MONTH_NAMES (dictionary)")
print("  • HIJRI_MONTH_NAMES_EN (dictionary)")

print("\n✅ تعطیلات و رویدادها:")
print("  • is_iranian_holiday()")
print("  • get_all_holidays_for_jalali_date()")
print("  • persian_events (166 رویداد)")
print("  • persian_events_en (166 رویداد)")
print("  • hijri_events (رویدادهای قمری)")
print("  • gregorian_events (رویدادهای میلادی)")

print("\n✅ سال کبیسه:")
print("  • is_leap_year_persian()")
print("  • is_hijri_leap()")

print("\n✅ تولید جدول (نیاز به pandas):")
print("  • DateDimensionGenerator")
print("  • DateRangeGenerator")

print("\n✅ اطلاعات امروز:")
print("  • CurrentDate")

print("\n"+"="*60)
print("🔗 لینک‌ها:")
print("  PyPI: https://pypi.org/project/multi-calendar-dimension/")
print("  GitHub: https://github.com/pouya-kia/multi-calendar-dimension")
print("  نویسنده: Pouya Kia (kiaa.pouya@gmail.com)")


📚 کتابخانه Multi-Calendar Dimension


✅ تبدیل تاریخ:
  • jalali_to_gregorian()
  • gregorian_to_jalali()
  • jalali_to_hijri()
  • gregorian_to_hijri()
  • hijri_to_gregorian()
  • hijri_to_jalali()

✅ نام ماه‌ها:
  • get_hijri_month_name()
  • HIJRI_MONTH_NAMES (dictionary)
  • HIJRI_MONTH_NAMES_EN (dictionary)

✅ تعطیلات و رویدادها:
  • is_iranian_holiday()
  • get_all_holidays_for_jalali_date()
  • persian_events (166 رویداد)
  • persian_events_en (166 رویداد)
  • hijri_events (رویدادهای قمری)
  • gregorian_events (رویدادهای میلادی)

✅ سال کبیسه:
  • is_leap_year_persian()
  • is_hijri_leap()

✅ تولید جدول (نیاز به pandas):
  • DateDimensionGenerator
  • DateRangeGenerator

✅ اطلاعات امروز:
  • CurrentDate

🔗 لینک‌ها:
  PyPI: https://pypi.org/project/multi-calendar-dimension/
  GitHub: https://github.com/pouya-kia/multi-calendar-dimension
  نویسنده: Pouya Kia (kiaa.pouya@gmail.com)
